In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
cd drive/My\ Drive/Work\&Study/stanford/coursework/cs236g/project/uda_gan/

/content/drive/My Drive/Work&Study/stanford/coursework/cs236g/project/uda_gan


In [3]:
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import time
import datetime, itertools
from torch.autograd import Function, Variable
import torchvision
import torch.utils.data as data
import torchvision.utils as vutils
import os
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import pickle
from scipy.signal import butter,lfilter,freqz,detrend,stft
from sklearn.utils import shuffle
from sklearn.metrics import f1_score
from Dann import *
from GRL import *
from helper_func import *
import gan_da_model as model
from gan_da_trainer import *
from Dataset import *
import random
# np.random.seed(1000)
# torch.backends.cudnn.deterministic = True
# torch.backends.cudnn.benchmark = False

# Form dataset 

In [4]:
# data path
filePath = '../../../../../CMU/research/Damage_diagnosis/transfer/dataset/'
# Knowledge transfer direction
b1b2 = True
# Which vehicle
v_n =10
# Load data
x_dann, label_l, label_s, label_d, label_f, _, _=load_feature(filePath,v_n,b1b2)
x_dann_test, label_l_test, label_s_test, label_d_test, label_f_test, label_bb, damage_ls=load_feature(filePath,v_n,b1b2)

In [5]:
batch_size = 16
# source domain data and labels
x_source,yl_source,ys_source,yd_source,yf_source=shuffle(x_dann[0:round(len(x_dann)/2)],label_l,label_s,label_d,label_f,random_state=0)
# target domain data and labels
x_target,yl_target,ys_target,yd_target,yf_target=shuffle(x_dann[round(len(x_dann)/2):],label_l,label_s,label_d,label_f,random_state=0)
# hold out validation set
hold_out_size = int(len(x_source)*0.2)

dataset_train_source = Dataset(x_source[:-hold_out_size],yl_source[:-hold_out_size],ys_source[:-hold_out_size],\
                             yd_source[:-hold_out_size], yf_source[:-hold_out_size], is_train = True)
dataset_test_source = Dataset(x_source[-hold_out_size:],yl_source[-hold_out_size:],ys_source[-hold_out_size:],\
                             yd_source[-hold_out_size:], yf_source[-hold_out_size:], is_train = False)
dataset_train_target = Dataset(x_target[:-hold_out_size],yl_target[:-hold_out_size],ys_target[:-hold_out_size],\
                             yd_target[:-hold_out_size], yf_target[:-hold_out_size], is_train = True)
dataset_test_target = Dataset(x_target[-hold_out_size:],yl_target[-hold_out_size:],ys_target[-hold_out_size:],\
                             yd_target[-hold_out_size:], yf_target[-hold_out_size:], is_train = False)

source_train = torch.utils.data.DataLoader(dataset_train_source,batch_size, shuffle=True)
source_train_ = torch.utils.data.DataLoader(dataset_train_source,batch_size, shuffle=True)
source_test = torch.utils.data.DataLoader(dataset_test_source,batch_size, shuffle=False)
target_train = torch.utils.data.DataLoader(dataset_train_target,batch_size, shuffle=True)
target_train_ = torch.utils.data.DataLoader(dataset_train_target,batch_size, shuffle=True)
target_test = torch.utils.data.DataLoader(dataset_test_target,batch_size, shuffle=False)

# Generate to adapt:
https://openaccess.thecvf.com/content_cvpr_2018/papers/Sankaranarayanan_Generate_to_Adapt_CVPR_2018_paper.pdf

## Train

In [ ]:
class Args:
  batchSize=16
  nz = 128 #size of the latent z vector
  ngf = 64 #Number of filters to use in the generator network
  ndf = 64 #Number of filters to use in the discriminator network
  nepochs = 3000 #number of epochs to train for
  lr = 0.0005 #learning rate, default=0.0002
  beta1 = 0.8 #beta1 for adam. default=0.5
  gpu = 1 #GPU to use, -1 for CPU training
  outf = 'results' #folder to output images and model checkpoints
  manualSeed = 1
  adv_weight = 0.1 #weight for adv loss
  lrd = 0.0001 #learning rate decay, default=0.0002
  alpha = 0.3 #multiplicative factor for target adv. loss

opt = Args()
# print(opt)

# Setting random seed
if opt.manualSeed is None:
    opt.manualSeed = random.randint(1, 10000)
print("Random Seed: ", opt.manualSeed)
random.seed(opt.manualSeed)
torch.manual_seed(opt.manualSeed)
if opt.gpu>=0:
    torch.cuda.manual_seed_all(opt.manualSeed)

# GPU/CPU flags
torch.backends.cudnn.benchmark = True
if torch.cuda.is_available() and opt.gpu == -1:
    print("WARNING: You have a CUDA device, so you should probably run with --gpu [gpu id]")
if opt.gpu>=0:
    os.environ['CUDA_VISIBLE_DEVICES'] = str(opt.gpu)

nclasses = 13

GTA_trainer = GTA(opt, nclasses, source_train, source_test, target_train)
GTA_trainer.train()

Random Seed:  1


/content/drive/My Drive/Work&Study/stanford/coursework/cs236g/project/uda_gan/gan_da_trainer.py:72: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  inputv, labelv = Variable(inputs.cuda(), volatile=True), Variable(labels.cuda())


2021-02-22 08:49:08.657863| Epoch: 50, Val Accuracy: 70.833333 %
Test Accuracy: 19.791667 %


/content/drive/My Drive/Work&Study/stanford/coursework/cs236g/project/uda_gan/gan_da_trainer.py:97: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  inputv, labelv = Variable(inputs, volatile=True), Variable(labels)


2021-02-22 08:49:57.055486| Epoch: 100, Val Accuracy: 87.500000 %
Test Accuracy: 27.083333 %
2021-02-22 08:50:44.102723| Epoch: 150, Val Accuracy: 85.416667 %
Test Accuracy: 28.333333 %
2021-02-22 08:51:31.396221| Epoch: 200, Val Accuracy: 82.291667 %
Test Accuracy: 27.083333 %
2021-02-22 08:52:19.439038| Epoch: 250, Val Accuracy: 86.458333 %
Test Accuracy: 23.125000 %
2021-02-22 08:53:07.600241| Epoch: 300, Val Accuracy: 85.416667 %
Test Accuracy: 27.291667 %
2021-02-22 08:53:55.397815| Epoch: 350, Val Accuracy: 88.541667 %
Test Accuracy: 24.166667 %
2021-02-22 08:54:43.055434| Epoch: 400, Val Accuracy: 87.500000 %
Test Accuracy: 30.208333 %
2021-02-22 08:55:30.139757| Epoch: 450, Val Accuracy: 84.375000 %
Test Accuracy: 25.833333 %
2021-02-22 08:56:17.554348| Epoch: 500, Val Accuracy: 84.375000 %
Test Accuracy: 28.958333 %
2021-02-22 08:57:04.639459| Epoch: 550, Val Accuracy: 83.333333 %
Test Accuracy: 28.333333 %
2021-02-22 08:57:52.209922| Epoch: 600, Val Accuracy: 87.500000 %
Test

## Test

In [ ]:
class Args:
  batchSize=100
  nz = 128 #size of the latent z vector
  ngf = 64 #Number of filters to use in the generator network
  ndf = 64 #Number of filters to use in the discriminator network
  gpu = 1 #GPU to use, -1 for CPU training
  checkpoint_dir = 'results/models'
  model_best = 0

opt = Args()

netF = model._netF(opt)
netC = model._netC(opt, nclasses)

if opt.model_best == 0: 
    netF_path = os.path.join(opt.checkpoint_dir, 'netF.pth')
    netC_path = os.path.join(opt.checkpoint_dir, 'netC.pth')
else:
    netF_path = os.path.join(opt.checkpoint_dir, 'model_best_netF.pth')
    netC_path = os.path.join(opt.checkpoint_dir, 'model_best_netC.pth')
    
netF.load_state_dict(torch.load(netF_path))
netC.load_state_dict(torch.load(netC_path))

if opt.gpu>=0:
    netF.cuda()
    netC.cuda()
    
# Testing

netF.eval()
netC.eval()
    
total = 0
correct = 0

for i, datas in enumerate(target_train):
    inputs,_,_,_,labels = datas
    if opt.gpu>=0:
        inputs, labels = inputs.cuda(), labels.cuda()
    inputv, labelv = Variable(inputs, volatile=True), Variable(labels)

    outC = netC(netF(inputv))
    _, predicted = torch.max(outC.data, 1)        
    total += labels.size(0)
    correct += ((predicted == labels.cuda()).sum())
    
test_acc = 100*float(correct)/total
print('Test Accuracy: %f %%' % (test_acc))

# DANN approach (not a GAN baseline)
https://arxiv.org/abs/2006.03641

In [ ]:
device='cuda'
model = Dann().to(device)
optimizer = optim.SGD(model.parameters(), lr= 0.001, momentum= 0.9)
criterion = nn.CrossEntropyLoss().to(device)

allepoch=500
CUDA_LAUNCH_BLOCKING=1
losses = []
accuracies = []
f1s = []
losses_save,accuracy_save,f1_save = test_epoch(0,100,source_train,source_test,\
                                               target_train,target_test,model,criterion)
losses = np.append(losses,losses_save)
accuracies=np.append(accuracies,accuracy_save)
f1s=np.append(f1s,f1_save)
for epoch in range(allepoch):
    len_dataloader = min(len(source_train), len(target_train))
    total_steps = allepoch * len(source_train)
    i = 0
    model.train()
    for batch_idx, (data_source, data_source_, \
                    data_target, data_target_) in enumerate(zip(source_train, \
                    source_train_, target_train, target_train_)):
        start_time = time.time()
        s_img, s_label_l, s_label_s,_,_ = data_source
        s_img_, s_label_l_, s_label_s_,_,_ = data_source_

        start_steps = epoch * len(source_train)

        p = float(i + start_steps) / total_steps
        diff = 1.
        alpha = 2. / (1. + np.exp(-10 * p)) - 1

        optimizer = optimizer_scheduler(optimizer, p)
        optimizer.zero_grad()

        batch_size = len(s_label_l)

        domain_label_s = torch.zeros(2*batch_size)
        domain_label_s = domain_label_s.long()

        # source domain
        input_s = (s_img.to(device),s_img_.to(device))
        a,b,c,d = model(input_s,alpha)
        latent_xl,latent_xq = d
        pred_dl, pred_dq = b
        err_s_label = criterion(a, s_label_l.to(device))
        err_s_label_s = criterion(c, s_label_s_.to(device))
        err_s_domain = criterion(torch.cat((pred_dl,pred_dq)), domain_label_s.to(device))

        # training model using target data
        t_img, t_label_l, t_label_s,_,_ = data_target
        t_img_, t_label_l_, t_label_s_,_,_ = data_target_
        domain_label_t = torch.ones(2*batch_size)
        domain_label_t = domain_label_t.long()
        input_t = (t_img.to(device),t_img_.to(device))
        _, bt, _,dt = model(input_t,alpha)
        latent_xl_t,latent_xq_t = dt
        pred_dlt, pred_dqt = bt
        err_t_domain = criterion(torch.cat((pred_dlt,pred_dqt)), domain_label_t.to(device))

        err = (err_s_label + err_s_label_s) + (err_s_domain + err_t_domain)
        err.backward()
        optimizer.step()

        i += 1
        losses_save,accuracy_save,f1_save = test_epoch(epoch,100,source_train,source_test,\
                                                      target_train,target_test,model,criterion)
    losses = np.vstack((losses,losses_save))
    accuracies=np.vstack((accuracies,accuracy_save))
    f1s=np.vstack((f1s,f1_save))

    if(epoch % 10 == 0):
        print('epoch:{},[{}/{}],s_label:{:.3f},s_label_s:{:.3f},s_domain:{:.3f},t_domain:{:.3f},time{}'.
                format(epoch, i, len_dataloader, float(err_s_label), float(err_s_label_s), float(err_s_domain),
                        float(err_t_domain), time.time() - start_time))

# model_dict = model.state_dict()
# exp_name = 'forward'
# torch.save(model_dict, "../../save/model/DA-LS-{}-{}".format(exp_name,rngseed))

# # Save at the end of training
# losses2save = {"Losses": losses, "Accuracy": accuracies, "F1": f1s}
# with open('../../save/losses/loss-{}-{}'.format(exp_name,rngseed), 'wb') as handle:
#     pickle.dump(losses2save, handle, protocol=pickle.HIGHEST_PROTOCOL)


Source Train set: Average loss: 1.3865, 1.6089
Source test set: Average loss: 1.3861, 1.6093
Target train set: Average loss: 1.3865, 1.6089
Target test set: Average loss: 1.3862, 1.6093
Source Train set: Average loss: 1.3865, 1.6089
Source test set: Average loss: 1.3861, 1.6091
Target train set: Average loss: 1.3864, 1.6088
Target test set: Average loss: 1.3861, 1.6091
Source Train set: Average loss: 1.3865, 1.6089
Source test set: Average loss: 1.3861, 1.6092
Target train set: Average loss: 1.3864, 1.6089
Target test set: Average loss: 1.3861, 1.6092
Source Train set: Average loss: 1.3865, 1.6089
Source test set: Average loss: 1.3861, 1.6092
Target train set: Average loss: 1.3864, 1.6089
Target test set: Average loss: 1.3861, 1.6092
Source Train set: Average loss: 1.3865, 1.6090
Source test set: Average loss: 1.3861, 1.6093
Target train set: Average loss: 1.3864, 1.6089
Target test set: Average loss: 1.3861, 1.6092
Source Train set: Average loss: 1.3865, 1.6090
Source test set: Averag

In [ ]:
import matplotlib.pyplot as plt

fig=plt.figure(figsize=(6,4))
losses = np.array(losses)
color = ['r','k','b','m']
for i in range(4):
    plt.plot(losses[:,i*2],c=color[i],ls='--')
    plt.plot(losses[:,i*2+1],c=color[i],ls='-')
lgd=plt.legend(['Source, train, localization','Source, train, quantification',\
           'Source, test, localization','Source, test, quantification',\
           'Target, train, localization','Target, train, quantification',\
           'Target, test, localization','Target, test, Quantification'],loc=2,bbox_to_anchor=(1, 1),fontsize=10)
plt.xlabel('# epoch',fontsize=12)
plt.ylabel('Loss',fontsize=12)

In [ ]:
accuracies = np.array(accuracies)
color = ['r','k','b','m']
norm = [390*0.8,390*0.2,390*0.8,390*0.2]
for i in range(4):
    plt.plot(accuracies[:,i*2]/norm[i],c=color[i],ls='--')
    plt.plot(accuracies[:,i*2+1]/norm[i],c=color[i],ls='-')
plt.legend(['S, train, L','S, train, Q',\
           'S, test, L','S, test, Q',\
           'T, train, L','T, train, Q',\
           'T, test, L','T, test, Q'])
plt.title('Accuracy')